In [1]:
import glob
import shutil
from os.path import basename, splitext, exists
from collections import Counter

import pandas as pd
import numpy as np

In [10]:
df_train = pd.read_csv('/home/alexandrearaujo/kaggle/landmark-recognition-challenge/data/train.csv')

In [3]:
index = df_train['landmark_id'].drop_duplicates().index
test = df_train.loc[index, :]['id'].head(100).values
test_target = df_train.loc[index, :]['landmark_id'].head(100).values

In [4]:
train = []
train_target = []

for x, group in df_train[df_train['landmark_id'].isin(test_target)].groupby('landmark_id'):
    train.extend(group.head(4)['id'].values)
    train_target.extend(group.head(4)['landmark_id'].values)
    
for x, group in df_train[~df_train['landmark_id'].isin(test_target)].groupby('landmark_id'):
    train.extend(group.head(4)['id'].values)
    train_target.extend(group.head(4)['landmark_id'].values)

In [5]:
valid_test = pd.DataFrame({'id': test, 'landmark_id': test_target})
valid_train = pd.DataFrame({'id': train, 'landmark_id': train_target})

In [6]:
# verify
def func(x):
    path = '/media/hdd2/alexandrearaujo/kaggle/landmark-recognition-challenge/train_rescale/{}.jpg'.format(x)
    return exists(path)

valid_train['exists'] = valid_train['id'].apply(func)
valid_test['exists'] = valid_test['id'].apply(func)

In [7]:
valid_train = valid_train[valid_train['exists']][['id', 'landmark_id']].copy()
valid_test = valid_test[valid_test['exists']][['id', 'landmark_id']].copy()

In [93]:
for img in valid_train['id'].values:
    src_path = '/media/hdd2/alexandrearaujo/kaggle/landmark-recognition-challenge/train_rescale/{}.jpg'.format(img)
    dst_path = '/home/alexandrearaujo/kaggle/landmark-retrieval-challenge/validation/train/{}.jpg'.format(img)
    shutil.copy(src_path, dst_path)

for img in valid_test['id'].values:
    src_path = '/media/hdd2/alexandrearaujo/kaggle/landmark-recognition-challenge/train_rescale/{}.jpg'.format(img)
    dst_path = '/home/alexandrearaujo/kaggle/landmark-retrieval-challenge/validation/test/{}.jpg'.format(img)
    shutil.copy(src_path, dst_path)

In [27]:
import pickle
from os.path import isfile, splitext, dirname

def pickle_load(path):
  """function to load pickle object"""
  with open(path, 'rb') as f:
    return pickle.load(f, encoding='latin1')

def _pickle_dump(file, path):
  """function to dump picke object"""
  with open(path, 'wb') as f:
    pickle.dump(file, f, -1)

def _get_new_name(path):
  """rename file if file already exist"""
  i = 0
  new_path = path
  while isfile(new_path):
    ext = splitext(path)[1]
    new_path = path.replace(ext, '_{}{}'.format(i, ext))
    i += 1
  return new_path

def pickle_dump(file, path, force=False):
  """dump a file without deleting an existing one"""
  if force:
    _pickle_dump(file, path)
  elif not force:
    new_path = _get_new_name(path)
    _pickle_dump(file, new_path)

In [11]:
path_train_sift = glob.glob('../validation/feature_train_sift/*')
path_test_sift = glob.glob('../validation/feature_test_sift/*')

train_filenames = []
for pack in path_train_sift:
    for filename, loc, desc in pickle_load(pack):
        train_filenames.append(filename)
        
test_filenames = []
for pack in path_test_sift:
    for filename, loc, desc in pickle_load(pack):
        test_filenames.append(filename)

In [16]:
len(test_filenames)

99

In [24]:
target = df_train[df_train['id'].isin(test_filenames)][['id', 'landmark_id']].set_index('id')['landmark_id'].to_dict()

In [28]:
pickle_dump(target, '/home/alexandrearaujo/kaggle/landmark-retrieval-challenge/validation/target.pkl')

In [29]:
target_train = df_train[df_train['id'].isin(train_filenames)][['id', 'landmark_id']].set_index('id')['landmark_id'].to_dict()

In [31]:
pickle_dump(target_train, '/home/alexandrearaujo/kaggle/landmark-retrieval-challenge/validation/target_train.pkl')

In [32]:
target_train

{'ecb8e6dbb07414ad': 11610,
 '8bf47a96200b3378': 7465,
 '772e2da18311a1dc': 6303,
 '62c591b77e53684c': 1665,
 '1eec2812e379fe21': 13495,
 '89d8404314fafb5e': 11242,
 'c0be6f814176f1d3': 11913,
 '8219aec57cae5819': 7573,
 '949fbbd1d3c52218': 1328,
 'a6dec5763fcbef37': 15,
 '29253b32c33ce79e': 9118,
 '095925478945fe26': 2980,
 '0c31d2412b0fb49e': 9872,
 'bb463133de941c8e': 14919,
 '39686fe8e95faf5e': 2658,
 '1de28bc73c22ba05': 5318,
 '91e6af2fd22af4d0': 14481,
 '5816daa9a1c1e22b': 11424,
 '5f1efa80fe4bfc1f': 5773,
 '98f22f992c378e98': 13868,
 '4c3ffbd45c1e6902': 9659,
 'e9b55d2cbbf2651f': 3009,
 '829a9bf9884ef2b0': 3920,
 'fe00b9446f451f93': 2453,
 '6143cc4dad882139': 2146,
 '210d4e357cc08125': 8830,
 '4f84860dcbd8077f': 6819,
 '4c5bb8b566b1282a': 294,
 '05fb43938499143e': 11055,
 '32cf44dc5516c79b': 11499,
 '6addf7c76959fcb5': 10075,
 '29f2783556b05184': 6718,
 'ddaf558f5a21136a': 4594,
 '9a121c1a42eea0ad': 2783,
 '01fae1b30fab5069': 3509,
 '0b4f7ff5dda3fd80': 889,
 '16d9b735ded5a34c': 

limit_output extension: Maximum message size of 10000 exceeded with 27276 characters

In [33]:
pickle_load('../submit/9_flatindex_sift_vlad/filenames_index_4.dump')

['34b7431170012311',
 '5bbf9a809a0bb14c',
 'aa6e1f69e47b24f2',
 '6e7e417836a04672',
 '5761f073b4cef573',
 '15761c91fad00c22',
 '5fa55b48f941d75b',
 '58129a52f6bb9d27',
 '095d1b7127f18c29',
 '40e3412b85d75bca',
 '29ce53183a9acd9f',
 'e7836eb18f5f6ca3',
 'f2037489920d82cb',
 '9ef2d932dca5d345',
 'a7f5203ffe505c80',
 '9adb44e28aabd55d',
 '7b33ecca2b629cd5',
 '8e7cfd70fd4a769d',
 '0627921d28cfcad7',
 'a893fff146346f11',
 '0caf663c1a55bd56',
 '2f54f4173fcfe693',
 'a32204a3206cc163',
 'a250274d053b7e3d',
 'cedd40410944a29c',
 'b1b1e3a01925aa6b',
 '7e23ab4a65789379',
 '8feea52c7f87f236',
 '238c40fc9ab67d4a',
 'b3956470a061d3f1',
 '90829485c36ecac6',
 'b9894bb7a56e70a5',
 'b9f341dbab9dc87a',
 '9086286c18dfcef6',
 'a4796fb4f78b2bf9',
 'bb53113c386bddf0',
 '727d0e2d9b588a25',
 '235f02bae605b2c4',
 '7b620a74d5996d98',
 'ef7494280e1ff7ee',
 '43569b340179eb56',
 '199876bf63444988',
 '52e190f0f481b05a',
 'f186754b42e08225',
 'db69939d6f52595e',
 'b95bea4285600678',
 'c7b9fbe27525ff0f',
 '07fd1800d25

limit_output extension: Maximum message size of 10000 exceeded with 21005 characters